In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

The following is a list of files and their description

File descriptions
+ sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
+ test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
+ sample_submission.csv - a sample submission file in the correct format.
+ items.csv - supplemental information about the items/products.
+ item_categories.csv  - supplemental information about the items categories.
+ shops.csv- supplemental information about the shops.

In [2]:
# Import data from 'all' folder
item_cats = pd.read_csv('../all/item_categories.csv')
items = pd.read_csv('../all/items.csv')
shops = pd.read_csv('../all/shops.csv')

sales_train = pd.read_csv('../all/sales_train.csv.gz', parse_dates = True, compression = 'gzip')

In [3]:
item_cats.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [4]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [5]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [6]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


The descriptions of each of the columns are listed below.

Data fields
+ **ID** - an Id that represents a (Shop, Item) tuple within the test set
+ **shop_id** - unique identifier of a shop
+ **item_id** - unique identifier of a product
+ **item_category_id** - unique identifier of item category
+ **item_cnt_day** - number of products sold. You are predicting a monthly amount of this measure
+ **item_price** - current price of an item
+ **date** - date in format dd/mm/yyyy
+ **date_block_num** - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
+ **item_name** - name of item
+ **shop_name** - name of shop
+ **item_category_name** - name of item category

The first thing I notice here is that the item and shop names aren't in English but that shouldn't be a problem. Second, the ``date`` column in **sales_train** is in an awkward format which needs to be handled immediately. With that in mind, the column ``date_block_num`` may be deemed unnecessary since it's correlated to the actual ``date`` column. When that is resolved, it may be useful to combine the ``item_category_id`` column into the **sales_train**, and _test.csv_ for that matter, data sets which could be useful when exploring the data. One more thing that comes to mind is to convert the ID columns to categorical variables since there shouldn't be any fractional ID numbers.

In [7]:
# Format 'date' column
sales_train['date'] = pd.to_datetime(sales_train['date'], format = '%d.%m.%Y')

In [8]:
item_id_cat_merged = pd.merge(items, item_cats, how='left', on='item_category_id')

item_id_cat_merged.head()

,item_name,item_id,item_category_id,item_category_name
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,Программы - Для дома и офиса (Цифра)
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD
4,***КОРОБКА (СТЕКЛО) D,4,40,Кино - DVD


In [9]:
sales_train_merge = pd.merge(sales_train, item_id_cat_merged, how='left', on='item_id')

sales_train_merge.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name
0,2013-01-02,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,2013-01-03,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,2013-01-05,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
3,2013-01-06,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
4,2013-01-15,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства


The objective of this problem is to forecast the total amount of products sold in every shop, specifically for the provided test set. It's to be noted that the list of shops and products change from month to month which needs to be accounted for.

In order to create predictions for the test set, the test set needs to be explored to see what information is provided. It's stated that these numbers are for November 2015 which is the next month succeeding the training set.

In [10]:
# Import the test set
sales_test = pd.read_csv('../all/test.csv.gz', compression = 'gzip')

In [12]:
sales_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [13]:
sales_test.describe()

,ID,shop_id,item_id
count,214200.000000,214200.000000,214200.000000
mean,107099.500000,31.642857,11019.398627
std,61834.358168,17.561933,6252.644590
min,0.000000,2.000000,30.000000
25%,53549.750000,16.000000,5381.500000
50%,107099.500000,34.500000,11203.000000
75%,160649.250000,47.000000,16071.500000
max,214199.000000,59.000000,22167.000000


It seems all that provided for the test set is the ``shop_id`` and ``item_id`` so adding more columns similar to those in **sales_train_merge** might be necessary. Those columns would be:
+ ``date`` or ``date_block_num``
+ ``item_category_id``
+ ``item_category_name``
+ ``item_name``

In [14]:
# Add columns to sales_test
sales_test_merge = pd.merge(sales_test, item_id_cat_merged, how = 'left', on='item_id')

sales_test_merge.head()

,ID,shop_id,item_id,item_name,item_category_id,item_category_name
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3
1,1,5,5320,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства
2,2,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3
3,3,5,5232,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360
4,4,5,5268,"Need for Speed [PS4, русская версия]",20,Игры - PS4


In [15]:
# Add date column(s)
sales_test_merge['date'] = pd.to_datetime('01-11-2015', format = '%d-%m-%Y')

sales_test_merge.head()

,ID,shop_id,item_id,item_name,item_category_id,item_category_name,date
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,2015-11-01
1,1,5,5320,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,2015-11-01
2,2,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,2015-11-01
3,3,5,5232,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,2015-11-01
4,4,5,5268,"Need for Speed [PS4, русская версия]",20,Игры - PS4,2015-11-01
